In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
import pandas as pd
import numpy as np

# 1. Load data

In [3]:
df_2009_2010 = pd.read_excel("online_retail_II.xlsx", sheet_name='Year 2009-2010')

In [4]:
df_2010_2011 = pd.read_excel("online_retail_II.xlsx", sheet_name='Year 2010-2011')

In [5]:
df = pd.concat([df_2009_2010, df_2010_2011])

In [6]:
df.shape

(1067371, 8)

# 2. Check data quality

data type

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 73.3+ MB


In [8]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

columns name

In [9]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

data type

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 73.3+ MB


check missing value

In [11]:
df.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [12]:
df = df.dropna()

check white space

In [13]:
def empty_or_whitespace_strings(df):
    string_columns = df.select_dtypes(include=['object']).columns
    empty_whitespace_check = df[string_columns].applymap(lambda x: isinstance(x, str) and x.isspace())

    count_empty_whitespace = empty_whitespace_check.sum()
    rows_with_whitespace_indices = empty_whitespace_check.any(axis=1)
    list_row_with_whilespace_indices = df.index[rows_with_whitespace_indices].tolist()

    return count_empty_whitespace, list_row_with_whilespace_indices

In [14]:
show_whilespace, indices_whilesapce = empty_or_whitespace_strings(df)

In [15]:
show_whilespace

Invoice        0
StockCode      0
Description    0
Country        0
dtype: int64

check unique value

In [16]:
for i in df.columns:
    print('Columns name: ', i)
    print('Unique value: ', df[i].unique())
    print('Count unique value: ', df[i].nunique())
    print('-'*10)

Columns name:  Invoice
Unique value:  [489434 489435 489436 ... 581585 581586 581587]
Count unique value:  44876
----------
Columns name:  StockCode
Unique value:  [85048 '79323P' '79323W' ... 23562 23561 23843]
Count unique value:  4646
----------
Columns name:  Description
Unique value:  ['15CM CHRISTMAS GLASS BALL 20 LIGHTS' 'PINK CHERRY LIGHTS'
 ' WHITE CHERRY LIGHTS' ... 'SILVER AND BLACK ORBIT NECKLACE'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']
Count unique value:  5299
----------
Columns name:  Quantity
Unique value:  [    12     48     24     10     18      3     16      4      2      6
      8      1     28     30     60     32     56      9     25     36
     20    -12     -6     -4    -24     -3     -2    576    288     27
     64    160      5     -1     72     50     54      7    800     11
     58    192     96    144    100    180    216     15    240    108
     33     90     81    -81    -48     17     -5     40     -8     -9
     13     21  

Quantity and Price > 0

In [17]:
df = df[df['Quantity'] > 0]

In [18]:
df = df[df['Price'] > 0]

In [19]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


check Description

In [20]:
description_counts = df.groupby('StockCode').agg(number_description=('Description', pd.Series.nunique)).reset_index()

In [21]:
multiple_descriptions = description_counts[description_counts['number_description'] > 1]

In [22]:
multiple_descriptions = multiple_descriptions.sort_values("number_description", ascending=False)

In [23]:
multiple_descriptions

,StockCode,number_description
1423,22345,4
1458,22384,4
1422,22344,4
2275,23236,4
2235,23196,4
...,...,...
1493,22425,2
1500,22432,2
1501,22433,2
1513,22445,2


In [24]:
df[df['StockCode'] == 22345]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
3763,489784,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,12,2009-12-02 11:48:00,0.85,13269.0,United Kingdom
7287,490017,22345,PARTY PIZZA DISH BLUE WHITE SPOT,6,2009-12-03 12:31:00,0.85,14159.0,United Kingdom
17494,490786,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,12,2009-12-08 11:23:00,0.85,15079.0,United Kingdom
81546,496882,22345,PARTY PIZZA DISH BLUE WHITE SPOT,1,2010-02-04 14:13:00,0.85,15167.0,United Kingdom
88014,497615,22345,PARTY PIZZA DISH BLUE WHITE SPOT,6,2010-02-11 11:59:00,0.85,14527.0,United Kingdom
...,...,...,...,...,...,...,...,...
526151,580652,22345,PARTY PIZZA DISH BLUE POLKADOT,144,2011-12-05 13:33:00,0.19,17511.0,United Kingdom
526490,580671,22345,PARTY PIZZA DISH BLUE POLKADOT,24,2011-12-05 14:26:00,0.19,15159.0,United Kingdom
530028,580771,22345,PARTY PIZZA DISH BLUE POLKADOT,6,2011-12-06 10:58:00,0.19,16015.0,United Kingdom
533144,581015,22345,PARTY PIZZA DISH BLUE POLKADOT,24,2011-12-07 09:35:00,0.19,13949.0,United Kingdom


In [25]:
def fix_duplicates_description(df):
    df['Description'] = df.groupby('StockCode')['Description'].transform('first')
    return df

In [26]:
df = fix_duplicates_description(df)

In [27]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


check Description (again)

In [28]:
description_counts = df.groupby('StockCode').agg(number_description=('Description', pd.Series.nunique)).reset_index()

In [29]:
multiple_descriptions = description_counts[description_counts['number_description'] > 1]

In [30]:
multiple_descriptions = multiple_descriptions.sort_values("number_description", ascending=False)

In [31]:
multiple_descriptions

,StockCode,number_description


In [32]:
df[df['StockCode'] == 22345]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
3763,489784,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,12,2009-12-02 11:48:00,0.85,13269.0,United Kingdom
7287,490017,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,6,2009-12-03 12:31:00,0.85,14159.0,United Kingdom
17494,490786,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,12,2009-12-08 11:23:00,0.85,15079.0,United Kingdom
81546,496882,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,1,2010-02-04 14:13:00,0.85,15167.0,United Kingdom
88014,497615,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,6,2010-02-11 11:59:00,0.85,14527.0,United Kingdom
...,...,...,...,...,...,...,...,...
526151,580652,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,144,2011-12-05 13:33:00,0.19,17511.0,United Kingdom
526490,580671,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,24,2011-12-05 14:26:00,0.19,15159.0,United Kingdom
530028,580771,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,6,2011-12-06 10:58:00,0.19,16015.0,United Kingdom
533144,581015,22345,PARTY PIZZA DISH BLUE+WHITE SPOT,24,2011-12-07 09:35:00,0.19,13949.0,United Kingdom


check StockCode

In [33]:
stockcode_counts = df.groupby('Description').agg(number_stockcode=('StockCode', pd.Series.nunique)).reset_index()

In [34]:
multiple_stockcode = stockcode_counts[stockcode_counts['number_stockcode'] > 1]

In [35]:
multiple_stockcode = multiple_stockcode.sort_values("number_stockcode", ascending=False)

In [36]:
multiple_stockcode

,Description,number_stockcode
2394,"METAL SIGN,CUPCAKE SINGLE HOOK",3
996,COLOURING PENCILS BROWN TUBE,3
2467,MODERN CHRISTMAS TREE CANDLE,2
4201,This is a test product.,2
3988,SQUARE CHERRY BLOSSOM CABINET,2
3324,RETRO PLASTIC POLKA TRAY,2
3322,RETRO PLASTIC DAISY TRAY,2
3321,RETRO PLASTIC 70'S TRAY,2
2922,PINK FLOWERS RABBIT EASTER,2
2913,PINK FLOCK GLASS CANDLEHOLDER,2


In [37]:
df[df['Description'] == 'METAL SIGN,CUPCAKE SINGLE HOOK']

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
7255,490017,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2009-12-03 12:31:00,1.25,14159.0,United Kingdom
7256,490017,82613B,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2009-12-03 12:31:00,1.25,14159.0,United Kingdom
7258,490017,82613C,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2009-12-03 12:31:00,1.25,14159.0,United Kingdom
13875,490503,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",3,2009-12-06 14:00:00,1.25,14548.0,United Kingdom
19293,490972,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",16,2009-12-08 16:40:00,1.25,14911.0,EIRE
...,...,...,...,...,...,...,...,...
477522,577058,82613C,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2011-11-17 14:29:00,1.25,18122.0,United Kingdom
477524,577058,82613B,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2011-11-17 14:29:00,1.25,18122.0,United Kingdom
477589,577062,82613C,"METAL SIGN,CUPCAKE SINGLE HOOK",8,2011-11-17 14:40:00,1.25,18221.0,United Kingdom
482589,577479,82613C,"METAL SIGN,CUPCAKE SINGLE HOOK",8,2011-11-20 11:34:00,1.25,14911.0,EIRE


In [38]:
def fix_duplicates_stockcode(df):
    df['StockCode'] = df.groupby('Description')['StockCode'].transform('first')
    return df

In [39]:
df = fix_duplicates_stockcode(df)

In [40]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


check StockCode (again)

In [41]:
stockcode_counts = df.groupby('Description').agg(number_stockcode=('StockCode', pd.Series.nunique)).reset_index()

In [42]:
multiple_stockcode = stockcode_counts[stockcode_counts['number_stockcode'] > 1]

In [43]:
multiple_stockcode = multiple_stockcode.sort_values("number_stockcode", ascending=False)

In [44]:
multiple_stockcode

,Description,number_stockcode


In [45]:
df[df['Description'] == 'METAL SIGN,CUPCAKE SINGLE HOOK']

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
7255,490017,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2009-12-03 12:31:00,1.25,14159.0,United Kingdom
7256,490017,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2009-12-03 12:31:00,1.25,14159.0,United Kingdom
7258,490017,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2009-12-03 12:31:00,1.25,14159.0,United Kingdom
13875,490503,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",3,2009-12-06 14:00:00,1.25,14548.0,United Kingdom
19293,490972,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",16,2009-12-08 16:40:00,1.25,14911.0,EIRE
...,...,...,...,...,...,...,...,...
477522,577058,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2011-11-17 14:29:00,1.25,18122.0,United Kingdom
477524,577058,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",1,2011-11-17 14:29:00,1.25,18122.0,United Kingdom
477589,577062,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",8,2011-11-17 14:40:00,1.25,18221.0,United Kingdom
482589,577479,82613A,"METAL SIGN,CUPCAKE SINGLE HOOK",8,2011-11-20 11:34:00,1.25,14911.0,EIRE


check unique value (again)

In [46]:
for i in df.columns:
    print('Columns name: ', i)
    print('Unique value: ', df[i].unique())
    print('Count unique value: ', df[i].nunique())
    print('-'*10)

Columns name:  Invoice
Unique value:  [489434 489435 489436 ... 581585 581586 581587]
Count unique value:  36969
----------
Columns name:  StockCode
Unique value:  [85048 '79323P' '79323W' ... 23562 23561 23843]
Count unique value:  4608
----------
Columns name:  Description
Unique value:  ['15CM CHRISTMAS GLASS BALL 20 LIGHTS' 'PINK CHERRY LIGHTS'
 ' WHITE CHERRY LIGHTS' ... 'SET OF 6 RIBBONS PERFECTLY PRETTY  '
 'SET OF 6 RIBBONS PARTY' 'PAPER CRAFT , LITTLE BIRDIE']


Count unique value:  4608
----------
Columns name:  Quantity
Unique value:  [   12    48    24    10    18     3    16     4     2     6     8     1
    28    30    60    32    56     9    25    36    20   576   288    27
    64   160     5    72    50    54     7   800    11    58   192    96
   144   100   180   216    15   240   108    33    90    81    17    40
    13    21   120   200   168    19   280   128    80   300   450    14
    61   480    46   504    45  4320  5184  4008    22   600   505   500
    23    47   432   252    75   360   107    49    42    66    34    52
   400    70   936    35   256  1500   320   130  1002    26  1080    29
    67   960   864   150   126   468   336  3000    38   208   334   124
   125   117    39    55    78    77   350   526    98   112    59    37
   408  1440   832   624   204   198   608   224    43    65   720   271
    41   264   225   104   163   187   312   164   110   250   284    89
   118  6012   866  6144  2040  2520  3888  5280

clean Description

In [47]:
df['Description'] = df['Description'].str.replace(',', '')

In [48]:
df['Description'] = df['Description'].str.replace('*', '')

In [49]:
df['Description'] = df['Description'].str.strip()

In [50]:
for n in range(2,11,1):
    df['Description'] = df['Description'].str.replace(' '*n, '')

In [51]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


# 3. Manipulate cost for optimization model

In [52]:
np.random.seed(42)
df['profit_margin'] = np.random.uniform(0.01, 0.5, size=len(df))

In [53]:
df['Cost_price'] = df['Price'] * (1 - df['profit_margin'])

# 4. Export cleaned data

In [54]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,profit_margin,Cost_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,0.193525,5.605004
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0.475850,3.538012
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0.368677,4.261430
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,0.303343,1.462980
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,0.086449,1.141939


In [55]:
df.to_csv('01. Cleaned_retail_data.csv', sep=',', index=False)